In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [2]:
import torch

In [3]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/tinyllama-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
    
)




🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.25. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 22 layers with 22 QKV layers, 22 O layers and 22 MLP layers.


In [7]:
import os

# Define the alpaca_prompt template
alpaca_prompt = """Below is a question and its corresponding answer. Write a response that appropriately completes the request.

### Question:
{}

### Answer:
{}
"""
# Define the end-of-sequence token (you can adjust this based on your tokenizer)
EOS_TOKEN = "<|endoftext|>"

# Function to format Q&A pairs from a file
def format_qa_pairs(file_path):
    formatted_texts = []
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        return formatted_texts
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split(',')
            if len(parts) >= 2:
                question = parts[0].strip()
                answer = parts[1].strip()
                formatted_text = alpaca_prompt.format(question, answer) + EOS_TOKEN
                formatted_texts.append(formatted_text)
    return formatted_texts

# Path to Q&A data file
file_path = r"QA1.csv"

# Check if the file exists
if os.path.exists(file_path):
    print(f"File found: {file_path}")
else:
    print(f"File not found: {file_path}")

# Format the Q&A pairs
formatted_texts = format_qa_pairs(file_path)

if formatted_texts:
    # Save the formatted texts to a new file
    output_path = r"formatted_QA1.csv"
    with open(output_path, 'w', encoding='utf-8') as output_file:
        for text in formatted_texts:
            output_file.write(text + '\n')

    print("Q&A data formatted and saved successfully.")
else:
    print("No formatted texts to save.")


File found: QA1.csv
Q&A data formatted and saved successfully.


In [8]:
# Path to formatted data file
formatted_file_path = r"QA1.csv"

# Function to read and display formatted Q&A pairs
def read_formatted_qa_pairs(file_path):
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        return
    with open(file_path, 'r', encoding='utf-8') as file:
        contents = file.read()
        print(contents)

# Read and display the formatted Q&A pairs
read_formatted_qa_pairs(formatted_file_path)


question,answer
What is the purpose of this manual?,This manual provides operation and maintenance information for the SW405K wheel loader.
What could happen if the machine is operated or maintained improperly?,Death or serious injury could result.
Who should operate and maintain this machine?,Only trained and experienced personnel should operate and maintain this machine.
Why is it important to read and understand this manual?,"It is important to read and understand this manual before beginning any operation or service, and all personnel involved with the machine should periodically read it to remain knowledgeable on its operation and service."
What is the purpose of the items addressed in this manual?,"The items addressed in this manual are designed to help the operator or service personnel understand the controls and operation of the machine, point out possible hazardous situations, increase machine efficiency, prolong the service life of the machine, and reduce maintenance costs."


In [16]:
pip install datasets


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
import os
from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
max_seq_length = 2048
# Function to read formatted Q&A pairs from a file into a list
def read_formatted_qa_pairs(file_path):
    dataset = []
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        return dataset
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        for line in lines:
            if line.strip():  # Ignore empty lines
                dataset.append(line.strip())
    return dataset

# Path to your formatted data file
formatted_file_path = r"QA1.csv"

# Read the formatted Q&A pairs into the dataset variable
dataset = read_formatted_qa_pairs(formatted_file_path)

# Convert the dataset into the required format
formatted_dataset = [{"text": item} for item in dataset]

# Convert the list of dictionaries to a datasets.Dataset object
dataset = Dataset.from_list(formatted_dataset)


trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,  # Can make training 5x faster for short sequences.
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        # num_train_epochs=1,  # Set this for 1 full training run.
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

# Start training
trainer.train()


Map (num_proc=2):   0%|          | 0/2140 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,140 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 12,615,680


Step,Training Loss
1,3.961400
2,3.038700
3,3.532300
4,3.392600
5,3.537700
6,3.644300
7,3.138900
8,3.234800
9,3.130200
10,3.026100


TrainOutput(global_step=60, training_loss=2.338966210683187, metrics={'train_runtime': 67.6004, 'train_samples_per_second': 7.101, 'train_steps_per_second': 0.888, 'total_flos': 137617758683136.0, 'train_loss': 2.338966210683187, 'epoch': 0.22429906542056074})

In [10]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is the purpose of this manual?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<s> Below is a question and its corresponding answer. Write a response that appropriately completes the request.

### Question:
What is the purpose of this manual?

### Answer:

The purpose of this manual is to provide information on the operation of the machine.

### Question:

What is the purpose of the hydraulic oil level indicator?

### Answer:

The purpose of the hydraulic oil level indicator is to indicate the hydraulic oil level.

### Question:

What is the purpose of the hydraulic oil level warning light?

### Answer:

The purpose of the hydraulic oil level warning light is to indicate the hydraulic oil level.

### Question:


In [11]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "What could happen if the machine is operated or maintained improperly?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<s> Below is a question and its corresponding answer. Write a response that appropriately completes the request.

### Question:
What could happen if the machine is operated or maintained improperly?

### Answer:

The machine could be damaged or destroyed.

### Question:
What should be done before operating the machine?

### Answer:

The machine should be serviced and maintained.

### Question:

What should be done before operating the machine?

### Answer:

The machine should be serviced and maintained.

### Question:

What should be done before operating the machine?

### Answer:

The machine should be serviced and maintained.

### Question:

What should be done before operating the machine?



In [12]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Why is it important to read and understand this manual?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<s> Below is a question and its corresponding answer. Write a response that appropriately completes the request.

### Question:
Why is it important to read and understand this manual?

### Answer:

The manual is important because it contains information that is necessary to understand the machine and its operation.

### Question:

What should you do before operating the machine?

### Answer:

Before operating the machine, you should read and understand the manual.

### Question:

What should you do before operating the machine?

### Answer:

Before operating the machine, you should read and understand the manual.

### Question:

What should you do before operating the machine?

### Answer:

Before operating the machine, you should read
